# Series temporales
Nuestra variable independiente será el tiempo que lo denotaremos por t $\to$ 
* ``si vamos al  pasado`` se expresará t-1, t-2, t-3,... 
* ``si vamos al  futuro`` se expresará t+1, t+2, t+3,... 

* Estacionaria --> 
    * ``se mantega la media`` 
    * `` sea homocedástica`` = se mantenga la varianza

In [ ]:
## Librerias
import pandas as pd
import numpy as np
from datetime import datetime
import seaborn as sns
import matplotlib.pyplot as plt
import statsmodels.api as sm
#from pandas_datareader import data as web
from pydataset import data
plt.style.use('ggplot')

In [ ]:
comienzo=datetime(2010,1,1)
final=datetime(2020,3,10)

In [ ]:
final-comienzo

In [ ]:
df = pd.read_csv('./Datos/Index2018.csv')

In [ ]:
df.head()

In [ ]:
df.describe()

In [ ]:
df.dtypes

In [ ]:
df.date=pd.to_datetime(df.date,dayfirst=True)

In [ ]:
df.dtypes

In [ ]:
df.head()

In [ ]:
df.spx.plot()

In [ ]:
df.nikkei.plot()

In [ ]:
df_fecha=df.copy()

In [ ]:
df_fecha.set_index("date",inplace=True)

In [ ]:
df_fecha.nikkei.plot()

In [ ]:
df_fecha.head()

In [ ]:
## Frecuencia diaria
df_fecha.asfreq('d')

In [ ]:
## Frecuencia último día del mes
df_fecha.asfreq('M')

In [ ]:
## Para rellenar los nulos
## usando forward fill
df_fecha.asfreq('d',method="ffill")

In [ ]:
## usando back fill
df_fecha.asfreq('d',method="bfill")

In [ ]:
## Frecuencia último día del mes laborable
df_fecha.asfreq('BM')

In [ ]:
df_fecha.asfreq('BM').isnull().sum()

In [ ]:
## Cierre de trimestre
df_fecha.asfreq('BQ')

#### Calcular tasas de variación por "Día"
* Tasa de Variación 

$\cfrac{VF-VI}{VI}=\cfrac{VF}{VI}-1$

## Ejercicio
Calcular la tasa de variación de cada una de las columnas generando una nueva columna

Sería el valor de la fila i partido el valor de la fila i-1 y despues restarlo por 1

In [ ]:
df["nikkei_lag1"]=df["nikkei"].shift(1)

In [ ]:
df_fecha["nikkei_tv"]=df_fecha["nikkei"]/df_fecha["nikkei"].shift(1) - 1

In [ ]:
df_fecha.head()

In [ ]:
def calculo_tv(df,columna):
    return df[columna]/df[columna].shift(1) - 1

In [ ]:
ciclo,tendencia=sm.tsa.filters.hpfilter(df["nikkei"])

In [ ]:
df["Tendencia_Nikkei"]=tendencia

In [ ]:
df[["nikkei","Tendencia_Nikkei"]].plot();

In [ ]:
df["nikkei_MV"]=df["nikkei"].rolling(5).mean()

In [ ]:
df[["nikkei","Tendencia_Nikkei","nikkei_MV"]].tail(20).plot();

In [ ]:
sm.tsa.seasonal_decompose(df["nikkei"].tail(60),model="additive",period=30).plot();

## machie learning
utilizabamos la función train_test_split y ``cogíamos`` una muestra aleatoria

In [ ]:
tamanyo=int(len(df)*0.7)
df_train=df.iloc[:tamanyo]
df_test=df.iloc[tamanyo:]

In [ ]:
## Estacionariedad
import statsmodels.tsa.stattools as sts

In [ ]:
sts.adfuller(df.nikkei)

$$ H_0: \text{Los datos son no estacionarios}$$
$$ H_1: \text{Los datos son estacionarios}$$


In [ ]:
print("El p valor es: {0:.2f}".format(sts.adfuller(df.nikkei)[1]))

In [ ]:
## Suavizado de datos
### Metodo de Holt-Winters

In [ ]:
from statsmodels.tsa.holtwinters import SimpleExpSmoothing

In [ ]:
cantidad=5
alpha=2/(cantidad+1)

In [ ]:
### otra manera de medias moviles
df["nikkei_5"]=df["nikkei"].ewm(alpha=alpha).mean()

In [ ]:
df.head()

In [ ]:
df[["nikkei","nikkei_5","nikkei_MV"]].tail(20).plot()

In [ ]:
df["nikkei_EXPS"]=SimpleExpSmoothing(df["nikkei"]).fit(smoothing_level=alpha).fittedvalues

In [ ]:
df[["nikkei","nikkei_5","nikkei_MV","nikkei_EXPS"]].tail(20).plot()

In [ ]:
## Doble suavizado
from statsmodels.tsa.holtwinters import ExponentialSmoothing

In [ ]:
df["nikkei_EXPS"]=ExponentialSmoothing(df["nikkei"]).fit(smoothing_level=alpha).fittedvalues.shift(-1)

In [ ]:
df[["nikkei","nikkei_5","nikkei_MV","nikkei_EXPS"]].tail(20).plot()

In [ ]:
## Suavizado triple
df["nikkei_EXPS"]=ExponentialSmoothing(df["nikkei"],trend="add",seasonal="add",seasonal_periods=12).fit(smoothing_level=alpha).fittedvalues

In [ ]:
df[["nikkei","nikkei_5","nikkei_MV","nikkei_EXPS"]].tail(20).plot()

In [ ]:
modelo=ExponentialSmoothing(df["nikkei"],trend="add",seasonal="add",seasonal_periods=12).fit()

In [ ]:
prediccion_20=modelo.forecast(20)

In [ ]:
df["nikkei"].tail(10).plot()
prediccion_20.plot(label="Prediccion modelo aditivo")

## Modelo AR
### AR(1)
$$y_t=\beta_1 y_{t-1} + C + \epsilon_{t-1}$$

### AR(p)
$$y_t=\beta_1 y_{t-1} + \beta_2 y_{t-2} + \ldots+\beta_p y_{t-p} + C + \epsilon_{t-1}$$

In [ ]:
from statsmodels.tsa.arima.model import ARIMA

In [ ]:
modelo_ar=ARIMA(df.nikkei,order=(1,0,0))
resultado_ar=modelo_ar.fit()

In [ ]:
resultado_ar.summary()

In [ ]:
modelo_ar2=ARIMA(df.nikkei,order=(2,0,0))
resultado_ar2=modelo_ar2.fit()
resultado_ar2.summary()

## Medias moviles
### MA(1)
$$ y_t = C + \epsilon_{t-1} $$

### MA(q)
$$ y_t = C + \epsilon_{t-1} + \ldots + \epsilon_{t-q}  $$

In [ ]:
modelo_ma=ARIMA(df.nikkei,order=(0,0,1))
resultado_ma=modelo_ma.fit()
resultado_ma.summary()

In [ ]:
### ARMA(1,1)
modelo_ma=ARIMA(df.nikkei,order=(1,0,1))
resultado_ma=modelo_ma.fit()
resultado_ma.summary()

In [ ]:
resultado_ar.summary()

In [ ]:
### ARMA(2,1)
modelo_arma21=ARIMA(df.nikkei,order=(3,0,3))
resultado_arma21=modelo_arma21.fit()
resultado_arma21.summary()

In [ ]:
### ARIMA(1,1,1)
modelo_arima111=ARIMA(df.nikkei,order=(1,1,1))
resultado_arima111=modelo_arima111.fit()
resultado_arima111.summary()

### Ejercicio
Del fichero uspopulation calcular el mejor modelo que se ajuste utilizando un modelo  ARIMA